In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from multicall import Call

from mainnet_launch.abis import CURVE_STABLESWAP_NG_ABI
from mainnet_launch.constants import *
from mainnet_launch.data_fetching.alchemy.get_events import fetch_events, get_each_event_in_contract
from plotly.subplots import make_subplots
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
    identity_with_bool_success,
    safe_normalize_6_with_bool_success,
    build_blocks_to_use,
)

import plotly.graph_objects as go

pio.templates.default = "plotly_white"

# https://etherscan.io/address/#readContract


from mainnet_launch.abis import ERC_20_ABI


def get_amount_staked_and_balance_of(autopool: AutopoolConstants, main_rewarder: str):

    contract = autopool.chain.client.eth.contract(autopool.autopool_eth_addr, abi=ERC_20_ABI)

    transfer_events = fetch_events(contract.events.Transfer, autopool.chain)
    to_wallets = transfer_events["to"].unique()
    print(autopool.name, "unique wallets:", len(to_wallets))
    # Create a list of balance calls for all unique wallet addresses
    # main_rewarder = '0x726104CfBd7ece2d1f5b3654a19109A9e2b6c27B'
    # note, might not be exact
    # Break calls into chunks of 500 to avoid request size limits
    chunk_size = 1000
    wallet_chunks = [to_wallets[i : i + chunk_size] for i in range(0, len(to_wallets), chunk_size)]

    staked_dfs = []
    balance_of_dfs = []

    for i, chunk in enumerate(wallet_chunks):
        print(f"Processing chunk {i+1}/{len(wallet_chunks)} with {len(chunk)} wallets")

        amount_staked_calls = [
            Call(
                target=main_rewarder,
                function=["balanceOf(address)(uint256)", addr],
                returns=[(addr, safe_normalize_with_bool_success)],
            )
            for addr in chunk
        ]

        balance_of_calls = [
            Call(
                target=autopool.autopool_eth_addr,
                function=["balanceOf(address)(uint256)", addr],
                returns=[(addr, safe_normalize_with_bool_success)],
            )
            for addr in chunk
        ]

        staked_chunk = get_raw_state_by_blocks(amount_staked_calls, build_blocks_to_use(autopool.chain), autopool.chain)
        balance_of_chunk = get_raw_state_by_blocks(
            balance_of_calls, build_blocks_to_use(autopool.chain), autopool.chain
        )

        staked_dfs.append(staked_chunk)
        balance_of_dfs.append(balance_of_chunk)
        print("success", "successfully fetched chunk", i + 1)

    # Merge all chunks together
    staked_df = pd.concat(staked_dfs, axis=1)
    balance_of_df = pd.concat(balance_of_dfs, axis=1)

    return staked_df, balance_of_df


# 8 minutes
auto_usd_staked, auto_usd_balance_of = get_amount_staked_and_balance_of(
    AUTO_USD, "0x726104CfBd7ece2d1f5b3654a19109A9e2b6c27B"
)
base_usd_staked, base_usd_balance_of = get_amount_staked_and_balance_of(
    BASE_USD, "0x4103A467166bbbDA3694AB739b391db6c6630595"
)
autoETH_staked, autoETH_balance_of = get_amount_staked_and_balance_of(
    AUTO_ETH, "0x60882D6f70857606Cdd37729ccCe882015d1755E"
)

/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/eth_abi/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


fetched logs from https://eth-mainnet 20,722,908 to 24,436,264 block_range 3,713,356 status: AchemyRequestStatus.SUCCESS num logs: 6313 2026-02-11T13:37:39.201653


/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest

autoUSD unique wallets: 391
Processing chunk 1/1 with 391 wallets
success successfully fetched chunk 1
fetched logs from https://base-mainnet 21,241,103 to 42,028,287 block_range 20,787,184 status: AchemyRequestStatus.SPLIT_RANGE_AND_TRY_AGAIN num logs: 0 2026-02-11T13:38:41.977188
fetched logs from https://base-mainnet 21,241,103 to 31,634,695 block_range 10,393,592 status: AchemyRequestStatus.SUCCESS num logs: 1453 2026-02-11T13:38:42.339723
fetched logs from https://base-mainnet 31,634,696 to 42,028,287 block_range 10,393,591 status: AchemyRequestStatus.SPLIT_RANGE_AND_TRY_AGAIN num logs: 0 2026-02-11T13:38:43.398630
fetched logs from https://base-mainnet 31,634,696 to 36,831,491 block_range 5,196,795 status: AchemyRequestStatus.SUCCESS num logs: 9998 2026-02-11T13:38:44.074246
fetched logs from https://base-mainnet 36,831,492 to 42,028,287 block_range 5,196,795 status: AchemyRequestStatus.SUCCESS num logs: 7268 2026-02-11T13:38:44.738779


/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest

baseUSD unique wallets: 2056
Processing chunk 1/3 with 1000 wallets
success successfully fetched chunk 1
Processing chunk 2/3 with 1000 wallets
success successfully fetched chunk 2
Processing chunk 3/3 with 56 wallets
success successfully fetched chunk 3
fetched logs from https://eth-mainnet 20,722,908 to 24,436,264 block_range 3,713,356 status: AchemyRequestStatus.SUCCESS num logs: 8938 2026-02-11T13:43:14.045811


/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest

autoETH unique wallets: 919
Processing chunk 1/1 with 919 wallets
success successfully fetched chunk 1


In [2]:
autoETH_price = 1.0245 * 1935
autoUSD_price = 1.068
base_usd_price = 1.0562

auto_usd_df = auto_usd_staked + auto_usd_balance_of
auto_usd_df = auto_usd_df[auto_usd_df.index > "2025-04-07"].copy()
auto_usd_df.index = auto_usd_df.index.date

base_usd_df = base_usd_staked + base_usd_balance_of
base_usd_df = base_usd_df[base_usd_df.index > "2025-05-22"].copy()
base_usd_df.index = base_usd_df.index.date

autoETH_df = autoETH_staked + autoETH_balance_of
autoETH_df = autoETH_df[autoETH_df.index > "2025-04-07"].copy()
autoETH_df.index = autoETH_df.index.date

auto_usd_df = auto_usd_df.drop(
    columns=[AUTO_USD.autopool_eth_addr, "0x726104CfBd7ece2d1f5b3654a19109A9e2b6c27B"], errors="ignore"
)
base_usd_df = base_usd_df.drop(
    columns=[BASE_USD.autopool_eth_addr, "0x4103A467166bbbDA3694AB739b391db6c6630595"], errors="ignore"
)

autoETH_df = autoETH_df.drop(
    columns=[AUTO_ETH.autopool_eth_addr, "0x60882D6f70857606Cdd37729ccCe882015d1755E"], errors="ignore"
)

autoETH_df = autoETH_df * autoETH_price
auto_usd_df = auto_usd_df * autoUSD_price
base_usd_df = base_usd_df * base_usd_price

full_df = pd.concat([autoETH_df, auto_usd_df, base_usd_df], axis=1).fillna(0)
full_df = full_df.T.groupby(level=0).sum().T

# Group columns by name and sum them together
current_users_df = full_df.loc[:, full_df.iloc[-1] > 0]

current_users_df

,0x000000000000000000000000000000000000dEaD,0x000000000004444c5dc75cB358380D2e3dE08A90,0x0000000f2eB9f69274678c76222B35eEc7588a65,0x00000d3b6B9B6732406ee993E9b768394e73b650,0x001D31846d08C23177011C6A523ED5B75823533E,0x0035Ee9Ddeaa0aabc5649dA02e492dA9De85A0df,0x00801b11586C8c00e67446abdd245917c6695375,0x00A385F8B49B2bf2f607eb4C861DaD75c2208faD,0x00C46d4F06198809E10AF095553FeF995195335F,0x01485ef10715F5B96BeFfA5552B0642E32C9aef6,...,0xfC3B7A21e65aee0e6959a524cE9Fbe9f8F4d95f3,0xfD56eE517bBce6A9598c14f01D16eb1293d16109,0xfD5BC91F691EC6AAfa609816F8da189C8BD18a00,0xfDC54e8E4395B8A5A7A55DA875d708151C798b61,0xfDb4ae5ea5C3Cf083F45721ae2732a84C6809540,0xfEac7B2BbbCDc103718E2929650Aea060f20440b,0xfFAe02C509864AD82B239059A6e44fEb800bAf33,0xfe45E9D56505bAE555F6Ff34A3149C86Bcfb5955,0xfeCa83500b78518191019Aac0BE755D064B3c079,0xff7Bb766bB3C918C7BDd0427b2c4E592B017F565
2025-04-07,0.10680,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,99372.635734,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1982.414931,0.000000,0.000000
2025-04-08,0.10680,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,99372.635734,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1982.414931,0.000000,0.000000
2025-04-09,0.10680,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,99372.635734,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1982.414931,0.000000,0.000000
2025-04-10,0.10680,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,99372.635734,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1982.414931,0.000000,0.000000
2025-04-11,0.10680,106.721456,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,99372.635734,...,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,1982.414931,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-06,0.21242,239.001250,6.654505e+06,21.040714,4945.377926,105628.824199,0.01015,122.674917,0.201533,190117.874876,...,106800.0,0.01017,127.847593,31958.891501,49.025502,3095.883126,7.292005e-15,1982.414931,0.000000,5542.249228
2026-02-07,0.21242,239.001250,6.654505e+06,21.040714,4945.377926,105628.824199,0.01015,122.674917,0.201533,190117.874876,...,106800.0,0.01017,127.847593,31958.891501,49.025502,3095.883126,7.292005e-15,1982.414931,0.000000,5542.249228
2026-02-08,0.21242,239.001250,6.654505e+06,21.040714,4945.377926,105628.824199,0.01015,122.674917,0.201533,190117.874876,...,106800.0,0.01017,127.847593,31958.891501,49.025502,3095.883126,7.292005e-15,1982.414931,250.067015,5542.249228
2026-02-09,0.21242,239.001250,6.654505e+06,21.040714,4945.377926,105628.824199,0.01015,122.674917,0.201533,190117.874876,...,106800.0,0.01017,127.847593,31958.891501,49.025502,3095.883126,7.292005e-15,1982.414931,250.067015,5542.249228


In [3]:
px.bar(full_df["0x0000000f2eB9f69274678c76222B35eEc7588a65"]).show()

In [4]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [6]:
# Get the last values for each address
last_values = current_users_df.iloc[-1]
# Filter to users who have had at least $100 at some point
last_values = last_values[current_users_df.max() > 100]
# Create buckets
buckets = {
    "< $250k": last_values[last_values < 250_000],
    "$250k - $500k": last_values[(last_values >= 250_000) & (last_values < 500_000)],
    "$500k - $1M": last_values[(last_values >= 500_000) & (last_values < 1_000_000)],
    "> $1M": last_values[last_values >= 1_000_000],
}

# Create summary dataframe
bucket_summary = pd.DataFrame(
    {
        "Bucket": buckets.keys(),
        "Count": [len(bucket) for bucket in buckets.values()],
        "Total Value": [bucket.sum() for bucket in buckets.values()],
        "Addresses": [bucket.index.tolist() for bucket in buckets.values()],
    }
)

bucket_summary.round(2)["Total Value"].sum()  # as of end of day feb 10th, 2026
bucket_summary["Total Value"] = bucket_summary["Total Value"].apply(lambda x: f"${x:,.0f}")
bucket_summary[["Bucket", "Count", "Total Value"]].to_clipboard()
bucket_summary[["Bucket", "Count", "Total Value"]]

,Bucket,Count,Total Value
0,< $250k,434,"$9,278,186"
1,$250k - $500k,19,"$6,419,848"
2,$500k - $1M,9,"$6,079,917"
3,> $1M,10,"$25,310,828"


In [11]:
def count_non_zero_days(series):
    non_zero = series[series > 0]
    return len(non_zero)


def calculate_longest_stretch(series):
    is_non_zero = (series > 0).astype(int)
    stretches = []
    current_stretch = 0

    for val in is_non_zero:
        if val == 1:
            current_stretch += 1
        else:
            if current_stretch > 0:
                stretches.append(current_stretch)
            current_stretch = 0

    if current_stretch > 0:
        stretches.append(current_stretch)

    return max(stretches) if stretches else 0


def get_first_non_zero_day(series):
    non_zero = series[series > 0]
    return non_zero.index[0] if len(non_zero) > 0 else None


def get_last_non_zero_day(series):
    non_zero = series[series > 0]
    return non_zero.index[-1] if len(non_zero) > 0 else None


def get_possible_days(series):
    first_non_zero = series[series > 0]
    if len(first_non_zero) == 0:
        return 0
    last_day = series.index[-1]
    first_non_zero_day = first_non_zero.index[0]
    return (last_day - first_non_zero_day).days + 1


def calculate_total_million_dollar_days(series):
    return series.sum() / 1_000_000


def calculate_dollar_days(series):
    return series.sum()


def calculate_last_value(series):
    return series.iloc[-1]


df = pd.DataFrame(
    {
        "address": full_df.columns,
        "num_non_zero_days": full_df.apply(count_non_zero_days),
        "longest_stretch": full_df.apply(calculate_longest_stretch),
        "possible_days": full_df.apply(get_possible_days),
        "first_non_zero_day": full_df.apply(get_first_non_zero_day),
        "last_non_zero_day": full_df.apply(get_last_non_zero_day),
        "median_size": full_df.median(),
        "mean_size": full_df.mean(),
        "max_size": full_df.max(),
        "total_million_dollar_days": full_df.apply(calculate_total_million_dollar_days),
        "total_dollar_days": full_df.apply(calculate_dollar_days),
        "last_value": full_df.apply(calculate_last_value),
    }
)
address_to_bucket = {}
for bucket_name, addresses in buckets.items():
    for address in addresses.index:
        address_to_bucket[address] = bucket_name

address_to_bucket

# off by at most 5.6% because we are using shares and not assets, but good enough for this analysis
# df = df[df["max_size"] > 1]
df["non_zero_over_possible"] = df["num_non_zero_days"] / df["possible_days"]
df["stretch_over_non_zero"] = df["longest_stretch"] / df["num_non_zero_days"]
df["entered_and_did_not_leave"] = df["non_zero_over_possible"] == 1
df["current_user"] = df["last_value"] > 0
df["bucket"] = df["address"].map(address_to_bucket).fillna("Not Current User")
df = df[df["max_size"] > 100].copy()  # non trivial

In [12]:
bucket_stats = df.groupby("bucket").agg(
    {
        "num_non_zero_days": ["median", "mean"],
        "entered_and_did_not_leave": lambda x: (x.sum() / len(x) * 100),
        "current_user": "sum",
        "address": "count",
    }
)

# Flatten column names
bucket_stats.columns = ["Median Days Active", "Mean Days Active", "% Never Fully Exited", "Current Users", "Count"]
bucket_stats.round(2).to_clipboard()
bucket_stats.round(2)

,Median Days Active,Mean Days Active,% Never Fully Exited,Current Users,Count
bucket,,,,,
$250k - $500k,263.0,234.37,73.68,19,19
$500k - $1M,190.0,202.11,88.89,9,9
< $250k,203.0,195.79,85.02,434,434
> $1M,221.0,201.00,70.00,10,10
Not Current User,27.0,55.32,0.00,0,1223


In [22]:
df["total_dollar_days"].sum()

np.float64(17713637631.1738)

In [ ]:
df["percent_of_total_dollar_days"] = df["total_dollar_days"] / df["total_dollar_days"].sum() * 100
# these 20 users are 50% of the total dollar days in autoUSD, baseUSD and autoETH. locking in 2k price for ETH. would change but prob not by muhc
df.sort_values("percent_of_total_dollar_days", ascending=False)[["address", "percent_of_total_dollar_days"]].head(20)

,address,percent_of_total_dollar_days
0x0000000f2eB9f69274678c76222B35eEc7588a65,0x0000000f2eB9f69274678c76222B35eEc7588a65,7.860435
0x53BEc1d713EFE6d97572043f5C559cfBfe4b9725,0x53BEc1d713EFE6d97572043f5C559cfBfe4b9725,6.630273
0x0c9AF4cB53191B48ef079b0D47bd88cba2965A7b,0x0c9AF4cB53191B48ef079b0D47bd88cba2965A7b,5.508317
0x8b4334d4812C530574Bd4F2763FcD22dE94A969B,0x8b4334d4812C530574Bd4F2763FcD22dE94A969B,4.234231
0xAbf61900C7e21afdDc0E36d49F16f49e016eCea9,0xAbf61900C7e21afdDc0E36d49F16f49e016eCea9,3.482589
0xE106167d35B7d5745b790828787d7452EAAf906A,0xE106167d35B7d5745b790828787d7452EAAf906A,3.431894
0xF2373F958069999248E34bCCc05B5614D865DDE6,0xF2373F958069999248E34bCCc05B5614D865DDE6,2.553569
0xccF343eeF0c5f2590EE30EFc9F564B33AEb3C7E6,0xccF343eeF0c5f2590EE30EFc9F564B33AEb3C7E6,1.911557
0xE094b53584D4Bb12fB0F8DF492fA872aC7e0fA96,0xE094b53584D4Bb12fB0F8DF492fA872aC7e0fA96,1.817077
0xa694bd1Eb17285b9e2cf117db971cD26324bC7Cd,0xa694bd1Eb17285b9e2cf117db971cD26324bC7Cd,1.391964


In [28]:
# Get the top 20 addresses by percent of total dollar days
top_20_addresses = df.sort_values("percent_of_total_dollar_days", ascending=False).head(20)["address"].tolist()

# Plot them as a bar chart
px.bar(full_df[top_20_addresses], title="Top 20 Users by % of Total Dollar Days")

In [ ]:
# 0x0000000f2eB9f69274678c76222B35eEc7588a65 7.8% lifetime ()

In [ ]:
df.sort_values("total_dollar_days", ascending=False)[["address", "total_dollar_days"]]

,address,num_non_zero_days,longest_stretch,possible_days,first_non_zero_day,last_non_zero_day,median_size,mean_size,max_size,total_million_dollar_days,total_dollar_days,last_value,non_zero_over_possible,stretch_over_non_zero,entered_and_did_not_leave,current_user,bucket
0x0000000f2eB9f69274678c76222B35eEc7588a65,0x0000000f2eB9f69274678c76222B35eEc7588a65,295,295,295,2025-04-22,2026-02-10,4.334479e+06,4.491513e+06,7.704910e+06,1392.369021,1.392369e+09,6.654505e+06,1.000000,1.0,True,True,> $1M
0x53BEc1d713EFE6d97572043f5C559cfBfe4b9725,0x53BEc1d713EFE6d97572043f5C559cfBfe4b9725,306,306,306,2025-04-11,2026-02-10,3.838113e+06,3.788589e+06,3.838113e+06,1174.462573,1.174463e+09,3.838113e+06,1.000000,1.0,True,True,> $1M
0x0c9AF4cB53191B48ef079b0D47bd88cba2965A7b,0x0c9AF4cB53191B48ef079b0D47bd88cba2965A7b,310,310,310,2025-04-07,2026-02-10,3.158941e+06,3.147494e+06,3.158941e+06,975.723263,9.757233e+08,3.158941e+06,1.000000,1.0,True,True,> $1M
0x8b4334d4812C530574Bd4F2763FcD22dE94A969B,0x8b4334d4812C530574Bd4F2763FcD22dE94A969B,310,310,310,2025-04-07,2026-02-10,2.712136e+06,2.419472e+06,2.765536e+06,750.036423,7.500364e+08,2.359696e+06,1.000000,1.0,True,True,> $1M
0xAbf61900C7e21afdDc0E36d49F16f49e016eCea9,0xAbf61900C7e21afdDc0E36d49F16f49e016eCea9,247,247,254,2025-06-02,2026-02-03,2.264382e+06,1.989978e+06,3.829879e+06,616.893279,6.168933e+08,0.000000e+00,0.972441,1.0,False,False,Not Current User
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x09b53c3AB5026B6270f3561488926f3af8A53aD4,0x09b53c3AB5026B6270f3561488926f3af8A53aD4,1,1,155,2025-09-09,2025-09-09,0.000000e+00,3.327953e-01,1.031665e+02,0.000103,1.031665e+02,0.000000e+00,0.006452,1.0,False,False,Not Current User
0xB1Dba499F322d4b6f5Ff9C4c1Ea78C6DfCc79741,0xB1Dba499F322d4b6f5Ff9C4c1Ea78C6DfCc79741,1,1,155,2025-09-09,2025-09-09,0.000000e+00,3.301784e-01,1.023553e+02,0.000102,1.023553e+02,0.000000e+00,0.006452,1.0,False,False,Not Current User
0x6220F18b6996a24C4f6E253c06A9532060a518dD,0x6220F18b6996a24C4f6E253c06A9532060a518dD,1,1,155,2025-09-09,2025-09-09,0.000000e+00,3.295857e-01,1.021716e+02,0.000102,1.021716e+02,0.000000e+00,0.006452,1.0,False,False,Not Current User
0x821F39316B90aA71cB201cC9485Fc5Ac779131Fc,0x821F39316B90aA71cB201cC9485Fc5Ac779131Fc,1,1,155,2025-09-09,2025-09-09,0.000000e+00,3.277763e-01,1.016107e+02,0.000102,1.016107e+02,0.000000e+00,0.006452,1.0,False,False,Not Current User


In [15]:
overall_stats = pd.DataFrame(
    {
        "Median Days Active": [df["num_non_zero_days"].median()],
        "Mean Days Active": [df["num_non_zero_days"].mean()],
        "% Never Fully Exited": [(df["entered_and_did_not_leave"].sum() / len(df) * 100)],
        "Current Users": [df["current_user"].sum()],
        "Count": [len(df)],
    }
)
overall_stats.round(2).to_clipboard()

In [14]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
median_all = df["num_non_zero_days"].median()
median_100k = df[df["max_size"] > 100_000]["num_non_zero_days"].median()
median_1m = df[df["max_size"] > 1_000_000]["num_non_zero_days"].median()

mean_all = df["num_non_zero_days"].mean()
mean_100k = df[df["max_size"] > 100_000]["num_non_zero_days"].mean()
mean_1m = df[df["max_size"] > 1_000_000]["num_non_zero_days"].mean()

# Add boolean columns for different size categories
df["is_over_100k"] = df["max_size"] > 100_000
df["is_over_1m"] = df["max_size"] > 1_000_000

# Calculate percentages for each category
entered_and_stayed_all = df["entered_and_did_not_leave"].value_counts(normalize=True)
entered_and_stayed_100k = df[df["is_over_100k"]]["entered_and_did_not_leave"].value_counts(normalize=True)
entered_and_stayed_1m = df[df["is_over_1m"]]["entered_and_did_not_leave"].value_counts(normalize=True)


summary_df = pd.DataFrame(
    {
        "Category": ["All Users", "Any point Max Size > 10k", "Any point Max Size > 100k", "Any point Max Size > 1M"],
        "Median Days": [median_all, df[df["max_size"] > 10_000]["num_non_zero_days"].median(), median_100k, median_1m],
        "Mean Days": [mean_all, df[df["max_size"] > 10_000]["num_non_zero_days"].mean(), mean_100k, mean_1m],
        "% Entered and Stayed": [
            entered_and_stayed_all.get(True, 0) * 100,
            df[df["max_size"] > 10_000]["entered_and_did_not_leave"].value_counts(normalize=True).get(True, 0) * 100,
            entered_and_stayed_100k.get(True, 0) * 100,
            entered_and_stayed_1m.get(True, 0) * 100,
        ],
    }
)

summary_df.round(1)
current_user_df = df[df["current_user"]]
median_current = current_user_df["num_non_zero_days"].median()
mean_current = current_user_df["num_non_zero_days"].mean()
entered_and_stayed_current = current_user_df["entered_and_did_not_leave"].value_counts(normalize=True)
summary_current_df = pd.DataFrame(
    {
        "Category": ["Current Users"],
        "Median Days": [median_current],
        "Mean Days": [mean_current],
        "% Entered and Stayed": [entered_and_stayed_current.get(True, 0) * 100],
    }
)
summary_df = pd.concat([summary_df, summary_current_df], ignore_index=True)
summary_df.round(2)  # .to_clipboard()

In [ ]:
df[(df["current_user"]) & (df["last_value"] > 400_000)]["address"]

In [ ]:
current_and_large = df[(df["current_user"]) & (df["max_size"] > 10_000)]
px.bar(full_df[current_and_large["address"].sample(min(10, len(current_and_large))).to_list()])

In [ ]:
px.ecdf(df["total_dollar_days"], title="Distribution of Total  Dollar Days")

In [ ]:
1_

In [ ]:
# Get the top 10 autoUSD holders by maximum balance
top_10_holders = auto_usd_df.sum().nlargest(20).index

px.bar(auto_usd_df[top_10_holders])

In [ ]:
# Get the top 10 autoUSD holders by maximum balance
top_10_holders = base_usd_df.sum().nlargest(20).index

px.bar(base_usd_df[top_10_holders])

In [ ]:
# there is double counting here not sure where but it it large for autoUSD

In [ ]:
px.line(auto_usd_df.sum(axis=1))

In [ ]:
total_dollar_days = auto_usd_df.sum().sum()  # close enought to treat as 1
large_area_under_curve = auto_usd_df.sum().sort_values().tail(100).index  # dollars * days staked
dollar_days_explained_by_top_100 = auto_usd_df[large_area_under_curve].sum().sum() / total_dollar_days
dollar_days_explained_by_top_100
# top 100 users explalins 94%
# top 50 is 85% of a dollar day

In [ ]:
a_person = "0x7E2337368E571F086c7467Ec3c320A3F98DF07F3"

px.line(base_usd_df[a_person], title="autoUSD whales shares staked over time")

In [ ]:
at_leak_100k_dollar_days = df[df["total_million_dollar_days"] >= 1]

In [ ]:
px.scatter(
    at_leak_100k_dollar_days,
    x="possible_days",
    y="num_non_zero_days",
    title="days with funds > 0 vs possible days to have funds > 0",
    color="total_million_dollar_days",
    hover_data=["address"],
)

In [ ]:
num_entered_and_never_fully_left = len(df[df["non_zero_over_possible"] == 1])
num_entered_and_never_fully_left / len(df)
# 20% of our users have added funds and never fully left, but they might have reduced their position significantly at some point. Let's see how many had more than 100k dollar days, which is a proxy for having a significant amount of funds staked for a significant amount of time.

In [ ]:
px.histogram(
    df["non_zero_over_possible"],
    nbins=10,
    title="Distribution of % of days with funds > 0 vs possible days to have funds > 0",
)

In [ ]:
px.histogram(df["num_non_zero_days"], title="Distribution of number of days with funds > 0")

In [ ]:
px.ecdf(df["total_million_dollar_days"], log_x=True)

In [ ]:
random_guy = df[df["non_zero_over_possible"] == 1].sample(1)["address"].iloc[0]

px.bar(base_usd_df["0x4bdE76720Ae8415491144745b10f2db91e3D520A"])

In [ ]:
px.histogram(df["total_million_dollar_days"], title="Distribution of Million Dollar Days per User", log_x=True)

In [ ]:
px.scatter(df, x="num_non_zero_days", y="longest_stretch", title="Million Dollar Days vs. Number of Non-Zero Days")

In [ ]:
px.histogram(df["stretch_over_non_zero"], title="Longest Non Zero Streak Distribution")

In [ ]:
df.columns

In [ ]:
px.histogram(df["longest_stretch"], title="Longest Non Zero Streak Distribution")

In [ ]:
px.histogram(df["longest_stretch"], title="Longest Non Zero Streak Distribution")

In [ ]:
px.line(df, x='total_million_dollar_days', '')

In [ ]:
# between the first and last chunk deposit, (last non zero value) how many days is it

# largest string of non zero days?

# given first non zero, and last non zero, how mnay of those days had funds?

In [ ]:
# need distribtuion of lengths of deposits
# there is this niche of our users who are like X.

In [ ]:
import plotly.express as px

# Get the max value for each column (wallet address) in the dataframe
max_values = auto_USD_amount_staked_df.max()

# Filter columns where max value is greater than 500000
large_holders = max_values[max_values > 500000].index.tolist()

print(f"Number of wallets with max balance > 500000: {len(large_holders)}")
print("\nWallet addresses:")
for wallet in large_holders:
    print(f"{wallet}: {max_values[wallet]:,.2f}")

In [ ]:
# autoUSD_main_rewarder = '0x726104cfbd7ece2d1f5b3654a19109a9e2b6c27b'

# balace of, amount staked,

In [ ]:
# # Feb 10, 2026
# # largest 5 baseUSD users
# https://debank.com/profile/0x0000000f2eb9f69274678c76222b35eec7588a65, 5.8M
# https://debank.com/profile/0x4516d35434fc081c9253a03a53510d3d52ca9b65, 1.3M
# https://debank.com/profile/0x24e9cbb9ddda1247ae4b4eeee3c569a2190ac401, 1.1M
# https://debank.com/profile/0xdb6e9e7390e9acc34619e56efa48ade01cff6f12, 1.0M
# https://debank.com/profile/0xccf343eef0c5f2590ee30efc9f564b33aeb3c7e6 .9M

total_base_usd = 18  # Million
top_5_users = sum([5.8, 1.3, 1.1, 1.0, 0.9])
percent_owned_by_top_5 = top_5_users / total_base_usd * 100
print(f"Top 5 users own {top_5_users} Million, {percent_owned_by_top_5:.2f}% of ${total_base_usd} Million baseUSD")

# # largest 5 autoUSD users
# https://debank.com/profile/0x15dc4c43b0d3283ae4f193bb8ac97181f4bbb56c 3M
# https://debank.com/profile/0x8b4334d4812c530574bd4f2763fcd22de94a969b 2.4M
# https://debank.com/profile/0x8d4d699abf3044306008c6bea8e78632f999c281 1.2M
# https://debank.com/profile/0x0000000f2eb9f69274678c76222b35eec7588a65 .9M
# https://debank.com/profile/0xd1d48ca1a09cb9a500c69041c63cc409be245de7 .9M

total_auto_usd = 15.8  # Million
top_5_auto_usd_users = sum([3, 2.4, 1.2, 0.9, 0.9])
percent_owned_by_top_5_auto_usd = top_5_auto_usd_users / total_auto_usd * 100
print(
    f"Top 5 users own {top_5_auto_usd_users} million {percent_owned_by_top_5_auto_usd:.2f}% of ${total_auto_usd} Million autoUSD"
)

# # autoETH
# https://debank.com/profile/0x53bec1d713efe6d97572043f5c559cfbfe4b9725 1.98k ETH
# https://debank.com/profile/0x0c9af4cb53191b48ef079b0d47bd88cba2965a7b 1.63K ETH
# https://debank.com/profile/0x051203d8cbd5bec7c146f2069732f2f702090bc6 .33k ETH
# https://debank.com/profile/0xa899578504b1443f9f3e474c26cb28332d8bae6a .28k ETH
# https://debank.com/profile/0xc4374c15b5d3e0f43125cc944abf5d1411a4a0db .23k ETH


total_autoETH = 8.8  # thousand
top_5_autoETH_users = sum([1.98, 1.63, 0.33, 0.28, 0.23])
percent_owned_by_top_5_autoETH = top_5_autoETH_users / total_autoETH * 100
print(
    f"Top 5 users own {top_5_autoETH_users} thousand {percent_owned_by_top_5_autoETH:.2f}% of {total_autoETH} thousand autoETH"
)


print(
    f"\n{'='*60}\n{'Stablecoin':<15} {'Total Supply':<15} {'Top 5 Holdings':<18} {'Concentration':<15}\n{'='*60}\n{'baseUSD':<15} ${total_base_usd}M{'':<9} ${top_5_users}M{'':<10} {percent_owned_by_top_5:.2f}%\n{'autoUSD':<15} ${total_auto_usd}M{'':<9} ${top_5_auto_usd_users}M{'':<10} {percent_owned_by_top_5_auto_usd:.2f}%\n{'='*60}"
)
print(
    f"{'autoETH':<15} {total_autoETH}k ETH{'':<6} {top_5_autoETH_users}k ETH{'':<7} {percent_owned_by_top_5_autoETH:.2f}%"
)

# 5 largest
# siloUSD total 8.7M, 8.7M owned by electric capital
# https://debank.com/profile/0x102b555161062ffddabdab1e4fe8d1e36482e383


# Infinifi
# 5M owned by one user
# https://debank.com/profile/0x92c3a5f226a47a44ad73f04663b305372cc665bf

# in total of total NAV

# 70M total NAV
8.7 + 5 + 10.1 + 8.4 + ((4500 * 1997) / 1_000_000)
# is 41 / 70M or 58% of total NAV
# 16 wallets (14 + 1 +1) are 58% of total NAV

In [ ]:
# # Feb 10, 2026
# # largest 5 baseUSD users
# https://debank.com/profile/0x0000000f2eb9f69274678c76222b35eec7588a65, 5.8M
# https://debank.com/profile/0x4516d35434fc081c9253a03a53510d3d52ca9b65, 1.3M
# https://debank.com/profile/0x24e9cbb9ddda1247ae4b4eeee3c569a2190ac401, 1.1M
# https://debank.com/profile/0xdb6e9e7390e9acc34619e56efa48ade01cff6f12, 1.0M
# https://debank.com/profile/0xccf343eef0c5f2590ee30efc9f564b33aeb3c7e6 .9M


# # largest 5 autoUSD users
# https://debank.com/profile/0x15dc4c43b0d3283ae4f193bb8ac97181f4bbb56c 3M
# https://debank.com/profile/0x8b4334d4812c530574bd4f2763fcd22de94a969b 2.4M
# https://debank.com/profile/0x8d4d699abf3044306008c6bea8e78632f999c281 1.2M
# https://debank.com/profile/0x0000000f2eb9f69274678c76222b35eec7588a65 .9M
# https://debank.com/profile/0xd1d48ca1a09cb9a500c69041c63cc409be245de7 .9M


# largest 5 autoETH users
# https://debank.com/profile/0x53bec1d713efe6d97572043f5c559cfbfe4b9725 1.98k ETH
# https://debank.com/profile/0x0c9af4cb53191b48ef079b0d47bd88cba2965a7b 1.63K ETH
# https://debank.com/profile/0x051203d8cbd5bec7c146f2069732f2f702090bc6 .33k ETH
# https://debank.com/profile/0xa899578504b1443f9f3e474c26cb28332d8bae6a .28k ETH
# https://debank.com/profile/0xc4374c15b5d3e0f43125cc944abf5d1411a4a0db .23k ETH


# only siloUSD
# siloUSD total 8.7M, 8.7M owned by electric capital
# https://debank.com/profile/0x102b555161062ffddabdab1e4fe8d1e36482e383


# only Infinifi USD
# 5M owned by one user
# https://debank.com/profile/0x92c3a5f226a47a44ad73f04663b305372cc665bf